<a href="https://colab.research.google.com/github/DT6A/Hands-On_Machine_Learning/blob/main/ML09_ANNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Perceptron

$h_w = step(w^Tx)$

$heaviside(z) = \begin{cases} 0, & \mbox{if } z < 0 \\ 1, & \mbox{if } z \geq 0 \end{cases}$

$sgn(z) = \begin{cases} -1, & \mbox{if } z < 0 \\ 0, & \mbox{if } z = 0, \\ 1, & \mbox{if } z > 0 \end{cases}$

$w_{i,  j} ^ {next\ step} = w_{i,  j} + \eta(y_j - \hat{y_j})x_i$

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)] # petal length and width
y = (iris.target == 0).astype(np.int) # Is Setosa

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])

In [ ]:
y_pred

array([1])

## Multi-Layer Perceptron

$\sigma(z) = 1 / (1 + e^{-z})$

$tanh(z) = 2\sigma(2z) - 1$

$ReLU(z) = max(0, z)$

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
X = mnist['data']
y = mnist['target']

In [ ]:
y = y.astype(np.int)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

###High-level API

In [ ]:
import tensorflow as tf

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, 
                                         feature_columns=feature_cols)
#dnn_clf = tf.contrib.learn.SKCompact(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)

W0621 17:29:41.292802 140046145337216 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0621 17:29:41.311930 140046145337216 estimator.py:453] Using temporary folder as model directory: /tmp/tmpozubkd5n
W0621 17:29:43.723870 140046145337216 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0621 17:29:43.932893 140046145337216 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/adagrad.py:76: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0621 17:29:44.017581 140046145337216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/estimators/head.py:678: ModelFnOps.__new__ (from tensorflow.contrib.learn.python.learn.estimator

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f5eaf62fb70>, 'hidden_units': [300, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x7f5ebd40cae8>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
y_p = []

for i in y_pred:
  y_p.append(i)

accuracy_score(y_p, y_test)

W0621 17:31:59.855468 140046145337216 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.


0.9781

###Low-level API

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 150
n_hidden3 = 70
n_outputs = 10
tf.reset_default_graph()

In [ ]:
X = tf.placeholder(tf.float32, (None, 28*28), name='X')
y = tf.placeholder(tf.int64, (None), name='y')

In [ ]:
def neuron_layer(X, n_neurons, name, activation=None):
  with tf.name_scope(name):
    n_inputs = int(X.get_shape()[1])
    stddev = 2 / (np.sqrt(n_inputs + n_neurons))
    init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
    W = tf.Variable(init, name='kernel')
    b = tf.Variable(tf.zeros([n_neurons]), name='bias')
    Z = tf.matmul(X, W) + b
    if activation is not None:
      return activation(Z)
    else:
      return Z

In [ ]:
with tf.name_scope('dnn'):
  hidden1 = neuron_layer(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
  hidden2 = neuron_layer(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
  logits = neuron_layer(hidden2, n_outputs, name='outputs')

In [ ]:
with tf.name_scope('dnn'):
  hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
  hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
  hidden3 = tf.layers.dense(hidden2, n_hidden3, name='hidden3', activation=tf.nn.relu)
  logits = tf.layers.dense(hidden3, n_outputs, name='outputs')

In [ ]:
with tf.name_scope('loss'):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
  loss = tf.reduce_mean(xentropy, name='loss')

In [ ]:
learning_rate = 0.01

with tf.name_scope('train'):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope('eval'):
  correct = tf.nn.in_top_k(logits, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data/')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
n_epochs = 20
batch_size = 50

In [ ]:
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
    acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
    
    print(epoch, 'Train accuracy:', acc_train, 'Val accuracy:', acc_val)
  save_path = saver.save(sess, './my_model_final/ckpt')    

0 Train accuracy: 0.86 Val accuracy: 0.9094
1 Train accuracy: 0.96 Val accuracy: 0.9298
2 Train accuracy: 0.92 Val accuracy: 0.9452
3 Train accuracy: 0.96 Val accuracy: 0.9468
4 Train accuracy: 0.98 Val accuracy: 0.955
5 Train accuracy: 0.88 Val accuracy: 0.9592
6 Train accuracy: 0.92 Val accuracy: 0.959
7 Train accuracy: 1.0 Val accuracy: 0.964
8 Train accuracy: 1.0 Val accuracy: 0.9676
9 Train accuracy: 0.98 Val accuracy: 0.9672
10 Train accuracy: 0.98 Val accuracy: 0.9688
11 Train accuracy: 1.0 Val accuracy: 0.9718
12 Train accuracy: 0.94 Val accuracy: 0.9702
13 Train accuracy: 0.98 Val accuracy: 0.9742
14 Train accuracy: 0.96 Val accuracy: 0.9742
15 Train accuracy: 1.0 Val accuracy: 0.9752
16 Train accuracy: 1.0 Val accuracy: 0.9748
17 Train accuracy: 1.0 Val accuracy: 0.9778
18 Train accuracy: 1.0 Val accuracy: 0.977
19 Train accuracy: 1.0 Val accuracy: 0.975


In [ ]:
new_img = mnist.test.next_batch(1)

In [ ]:
with tf.Session() as sess:
  saver.restore(sess, save_path)
  X_new_scaled = new_img[0]
  Z = logits.eval(feed_dict={X: X_new_scaled})
  y_pred = np.argmax(Z, axis=1)

In [ ]:
y_pred, new_img[1]

(array([7]), array([7], dtype=uint8))

In [ ]:
from sklearn.metrics import precision_score

with tf.Session() as sess:
  saver.restore(sess, save_path)
  data_test = mnist.test.next_batch(5000)
  Z = logits.eval(feed_dict={X: np.array(data_test[0])})
  y_pred = np.argmax(Z, axis=1)

In [ ]:
precision_score(data_test[1], y_pred, average=None).mean()

0.9755771316847245

In [ ]:
data_test[1]

array([2, 9, 6, ..., 1, 5, 4], dtype=uint8)